# Taller 9

### Aprendizaje en Redes Bayesianas: Estructura

##### Analítica Computacional para la Toma de Decisiones

---

|     Nombres      |      Apellidos       |     Login     |  Codigo   |
| :--------------: | :------------------: | :-----------: | :-------: |
|     Santiago     | Gonzalez Montealegre | s.gonzalez35  | 202012274 |
| Juliana Carolina |  Cardenas Barragan   | jc.cardenasb1 | 202011683 |

## Librerías Requeridas

In [1]:
import pandas as pd
from pgmpy.estimators import PC
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.estimators import HillClimbSearch
from pgmpy.estimators import K2Score
from pgmpy.estimators import BicScore
import numpy as np

## Estimando la estructura de un modelo a partir de datos: Restricciones

### 1

In [2]:
df = pd.read_csv("Data/data_asia.csv")
df.head()

,Unnamed: 0,asia,tub,smoke,lung,bronc,either,xray,dysp
0,0,no,no,no,no,yes,no,no,no
1,1,no,no,no,yes,no,yes,yes,no
2,2,no,no,no,no,no,no,no,no
3,3,no,no,yes,no,yes,no,no,yes
4,4,no,no,yes,no,no,no,no,no


In [3]:
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()

,asia,tub,smoke,lung,bronc,either,xray,dysp
0,no,no,no,no,yes,no,no,no
1,no,no,no,yes,no,yes,yes,no
2,no,no,no,no,no,no,no,no
3,no,no,yes,no,yes,no,no,yes
4,no,no,yes,no,no,no,no,no


In [4]:
df.describe()

,asia,tub,smoke,lung,bronc,either,xray,dysp
count,100000,100000,100000,100000,100000,100000,100000,100000
unique,2,2,2,2,2,2,2,2
top,no,no,no,no,no,no,no,no
freq,99057,98917,50054,94501,55093,93477,88991,56376


In [5]:
df.columns

Index(['asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp'], dtype='object')

### 2

In [6]:
est = PC(data=df)
estimated_model = est.estimate(variant="stable", max_cond_vars=4)

  0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
print(estimated_model)
print(estimated_model.nodes())
print(estimated_model.edges())

DAG with 8 nodes and 8 edges
['either', 'xray', 'bronc', 'dysp', 'tub', 'lung', 'smoke', 'asia']
[('either', 'xray'), ('either', 'dysp'), ('bronc', 'dysp'), ('tub', 'either'), ('lung', 'either'), ('smoke', 'bronc'), ('smoke', 'lung'), ('asia', 'tub')]


### 3

In [8]:
estimated_model = BayesianNetwork(estimated_model)
estimated_model.fit(data=df, estimator=MaximumLikelihoodEstimator)

for i in estimated_model.nodes():
    print(estimated_model.get_cpds(i))

+-------------+----------+----------+-----------+-----------+
| lung        | lung(no) | lung(no) | lung(yes) | lung(yes) |
+-------------+----------+----------+-----------+-----------+
| tub         | tub(no)  | tub(yes) | tub(no)   | tub(yes)  |
+-------------+----------+----------+-----------+-----------+
| either(no)  | 1.0      | 0.0      | 0.0       | 0.0       |
+-------------+----------+----------+-----------+-----------+
| either(yes) | 0.0      | 1.0      | 1.0       | 1.0       |
+-------------+----------+----------+-----------+-----------+
+-----------+----------------------+---------------------+
| either    | either(no)           | either(yes)         |
+-----------+----------------------+---------------------+
| xray(no)  | 0.9506402644500787   | 0.01962287291123716 |
+-----------+----------------------+---------------------+
| xray(yes) | 0.049359735549921374 | 0.9803771270887628  |
+-----------+----------------------+---------------------+
+------------+---------------

## Estimando la estructura de un modelo a partir de datos: Puntajes

### 1

In [9]:
scoring_method = K2Score(data=df)
esth = HillClimbSearch(data=df)
estimated_modelh = esth.estimate(
    scoring_method=scoring_method,
    max_indegree=4,
    max_iter=int(1e4)
    )

  0%|          | 0/10000 [00:00<?, ?it/s]

In [10]:
print(estimated_modelh)
print(estimated_modelh.nodes())
print(estimated_modelh.edges())

DAG with 8 nodes and 9 edges
['asia', 'tub', 'smoke', 'lung', 'bronc', 'either', 'xray', 'dysp']
[('asia', 'tub'), ('tub', 'either'), ('tub', 'dysp'), ('smoke', 'bronc'), ('lung', 'either'), ('lung', 'dysp'), ('lung', 'smoke'), ('bronc', 'dysp'), ('either', 'xray')]


### 2

In [11]:
print(scoring_method.score(estimated_modelh))

-223501.6696113951


### 3

In [12]:
# Repetir

## Ahora con Otra Red y Datos

### 1

In [13]:
def annual_growth(row, years):
    min_year = years["min"]
    max_year = years["max"]
    row["Indicator Name"] = row["Indicator Name"] + " - [annual growth %]"
    for year in range(max_year, min_year, -1):
        if not np.isnan(row[str(year)]) and not np.isnan(row[str(year - 1)]):
            row[str(year)] = 100 * (float(row[str(year)]) - float(row[str(year - 1)])) / abs(float(row[str(year - 1)]))
        else:
            row[str(year)] = np.nan
            row[str(min_year)] = np.nan
    return row

years = {"min" : 1960, "max" : 2019}
df_raw = pd.read_csv("Data/italy-raw-data.csv")
df_raw_growth = pd.DataFrame(data=[row if "growth" in row["Indicator Name"] else annual_growth(row, years) for index, row in df_raw.iterrows()])

print("There are " + str(df_raw_growth.shape[0]) + " indicators in the dataframe.")

There are 10 indicators in the dataframe.


In [14]:
print(df_raw_growth.head())

  Country Name Country Code  \
0        Italy          ITA   
1        Italy          ITA   
2        Italy          ITA   
3        Italy          ITA   
4        Italy          ITA   

                                      Indicator Name     Indicator Code  \
0                       Population growth (annual %)        SP.POP.GROW   
1                 Urban population growth (annual %)        SP.URB.GROW   
2                   GDP per capita growth (annual %)  NY.GDP.PCAP.KD.ZG   
3  Energy use (kg of oil equivalent per capita) -...  EG.USE.PCAP.KG.OE   
4  Fossil fuel energy consumption (% of total) - ...  EG.USE.COMM.FO.ZS   

       1960      1961       1962       1963      1964      1965  ...  \
0  1.993928  0.668383   0.676623   0.729553  0.822624  0.842109  ...   
1  2.836401  1.498807   1.506833   1.551287  1.636027  1.642485  ...   
2       NaN  7.486419   5.487478   4.842052  1.955533  2.402046  ...   
3       NaN       NaN  13.064053  11.188621  9.110076  7.753922  ...   
4 

### 2

In [15]:
nodes = ['Pop', 'Urb', 'GDP', 'EC', 'FFEC', 'REC', 'EI', 'CO2', 'CH4', 'N2O']
df_growth = df_raw_growth.transpose().iloc[4:]
df_growth.columns = nodes

print(df_growth.head(10))

           Pop       Urb       GDP         EC      FFEC  REC        EI  \
1960  1.993928  2.836401       NaN        NaN       NaN  NaN       NaN   
1961  0.668383  1.498807  7.486419        NaN       NaN  NaN       NaN   
1962  0.676623  1.506833  5.487478  13.064053  1.933224  NaN  5.753948   
1963  0.729553  1.551287  4.842052  11.188621 -0.167728  NaN  2.519427   
1964  0.822624  1.636027  1.955533   9.110076  1.075163  NaN  0.631028   
1965  0.842109  1.642485  2.402046   7.753922 -0.074481  NaN  2.335057   
1966  0.777304  1.568113  5.164163   8.706029  0.552096  NaN  1.518473   
1967  0.723778   1.50361  6.405678   8.416381   1.19174  NaN  1.938004   
1968  0.631737  1.403955  5.873595    9.46868  0.880322  NaN  1.596269   
1969  0.566059  1.326087   5.49918   7.893911  0.940778  NaN  0.725698   

            CO2  CH4  N2O  
1960        NaN  NaN  NaN  
1961        NaN  NaN  NaN  
1962  17.588694  NaN  NaN  
1963   12.51158  NaN  NaN  
1964    6.78298  NaN  NaN  
1965   7.848449  

### 3

In [16]:
TIERS_NUM = 3

def boundary_str(start, end, tier):
    return f'{tier}: {start:+0,.2f} to {end:+0,.2f}'

def relabel_value(v, boundaries):
    if v >= boundaries[0][0] and v <= boundaries[0][1]:
        return boundary_str(boundaries[0][0], boundaries[0][1], tier='A')
    elif v >= boundaries[1][0] and v <= boundaries[1][1]:
        return boundary_str(boundaries[1][0], boundaries[1][1], tier='B')
    elif v >= boundaries[2][0] and v <= boundaries[2][1]:
        return boundary_str(boundaries[2][0], boundaries[2][1], tier='C')
    else:
        return np.nan

def relabel(values, boundaries):
    result = []
    for v in values:
        result.append(relabel_value(v, boundaries))
    return result

def get_boundaries(tiers):
    prev_tier = tiers[0]
    boundaries = [(prev_tier[0], prev_tier[prev_tier.shape[0] - 1])]
    for index, tier in enumerate(tiers):
        if index != 0:
            boundaries.append((prev_tier[prev_tier.shape[0] - 1], tier[tier.shape[0] - 1]))
            prev_tier = tier
    return boundaries

new_columns = {}
for i, content in enumerate(df_growth.items()):
    (label, series) = content
    values = np.sort(np.array([x for x in series.tolist() if not np.isnan(x)] , dtype=float))
    if values.shape[0] < TIERS_NUM:
        print(f'Error: there are not enough data for label {label}')
        break
    boundaries = get_boundaries(tiers=np.array_split(values, TIERS_NUM))
    new_columns[label] = relabel(series.tolist(), boundaries)

df = pd.DataFrame(data=new_columns)
df.columns = nodes
df.index = range(years["min"], years["max"] + 1)

df.head(10)

,Pop,Urb,GDP,EC,FFEC,REC,EI,CO2,CH4,N2O
1960,C: +0.50 to +1.99,C: +0.82 to +2.84,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1961,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1963,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,B: -0.44 to +0.15,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1964,C: +0.50 to +1.99,C: +0.82 to +2.84,B: +1.24 to +2.89,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,B: -0.40 to +0.73,C: +3.81 to +17.59,NaN,NaN
1965,C: +0.50 to +1.99,C: +0.82 to +2.84,B: +1.24 to +2.89,C: +2.77 to +13.06,B: -0.44 to +0.15,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1966,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1967,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1968,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,C: +0.73 to +5.75,C: +3.81 to +17.59,NaN,NaN
1969,C: +0.50 to +1.99,C: +0.82 to +2.84,C: +2.89 to +7.49,C: +2.77 to +13.06,C: +0.15 to +2.11,NaN,B: -0.40 to +0.73,C: +3.81 to +17.59,NaN,NaN


### 6

In [18]:
model = BayesianNetwork([
    ('Pop', 'EC'),
    ('Urb', 'EC'),
    ('GDP', 'EC'),
    ('EC', 'FFEC'),
    ('EC', 'REC'),
    ('EC', 'EI'),
    ('REC', 'CO2'),
    ('REC', 'CH4'),
    ('REC', 'N2O'),
    ('FFEC', 'CO2'),
    ('FFEC', 'CH4'),
    ('FFEC', 'N2O'),
    ])